### Step 1: Load the packages

In [196]:
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats as stats
from statsmodels.stats.power import TTestIndPower
import matplotlib.pyplot as plt
%matplotlib inline
from google.cloud import bigquery
from google.cloud import bigquery_storage
import datetime as dt
from datetime import date, timedelta
import re
from math import ceil
import warnings
warnings.filterwarnings(action = 'ignore') # Suppresses pandas warnings
from IPython.display import display
from tqdm.notebook import tqdm_notebook # Displays progress bars when querying data from BQ

### Define some input parameters

In [304]:
# INPUT
query_path = 'G:\My Drive\APAC\Loved Brands Concept\Loved Brands Avenues KW\dbdf_tt_tier_calc_query.sql'
entity_id_var = "'TB_KW'"
asa_id_var = '20'
vertical_type_var = "['restaurants']"
start_date_var = 'DATE_TRUNC(DATE_SUB(CURRENT_DATE(), INTERVAL 14 DAY), WEEK)'
end_date_var = 'DATE_SUB(DATE_TRUNC(CURRENT_DATE(), WEEK), INTERVAL 1 DAY)' # Last two completed weeks
min_travel_time_var = '0'
max_travel_time_var = '100'
included_variants_var = "['Original', 'Control']"
included_assignment_types_var = "['Experiment', 'Automatic scheme', 'Manual']"
scheme_id_var = '463'
is_lb_var = "TRUE" # or you can also type in is_LB = 'Y' or is_LB = 'N'

### Step 2.1: Define the query

In [338]:
sql_query = """
-- Get the vendor IDs
WITH selected_vendors AS ( -- All vendors (LBs and non-LBs)
  SELECT DISTINCT a.entity_id, a.vendor_code, a.is_LB
  FROM `dh-logistics-product-ops.pricing.final_vendor_list_all_data_loved_brands_scaled_code` a
  LEFT JOIN `fulfillment-dwh-production.curated_data_shared_central_dwh.vendors` b ON a.entity_id = b.global_entity_id AND a.vendor_code = b.vendor_id
  WHERE TRUE
    AND entity_id = {entity_id_var} 
    AND asa_id = {asa_id_var} 
    AND update_timestamp = (SELECT MAX(update_timestamp) FROM `dh-logistics-product-ops.pricing.final_vendor_list_all_data_loved_brands_scaled_code`)
    AND b.vertical_type IN UNNEST({vertical_type_var})
    AND {is_lb_var}
),

scheme_config AS (
  SELECT *
  FROM `dh-logistics-product-ops.pricing.df_tiers_per_price_scheme_loved_brands_scaled_code` -- Could sometimes be out of date (always check)
  WHERE entity_id = {entity_id_var} AND asa_id = {asa_id_var} AND scheme_id = {scheme_id_var}
),

order_data AS (
    SELECT
      created_date,
      platform_order_code,
      dps.vendor_id,
      v.is_LB,
      dps.travel_time,
      dps.dps_travel_time_fee_local,
      dps.delivery_fee_local,
      dps.gfv_local,
      dps.gmv_local,
      CASE WHEN dps.entity_id IN ('TB_IQ', 'TB_OM', 'TB_AE', 'TB_KW', 'TB_QA', 'TB_JO', 'TB_BH') THEN tb.commission_amount_lc ELSE dps.commission_local END AS commission_local,
      dps.joker_vendor_fee_local,
      dps.mov_customer_fee_local,
      dps.service_fee_local,
      
      (CASE WHEN dps.is_delivery_fee_covered_by_discount = TRUE OR dps.is_delivery_fee_covered_by_voucher = TRUE THEN 0 ELSE dps.delivery_fee_local END)
      + CASE WHEN dps.entity_id IN ('TB_IQ', 'TB_OM', 'TB_AE', 'TB_KW', 'TB_QA', 'TB_JO', 'TB_BH') THEN tb.commission_amount_lc ELSE dps.commission_local END
      + dps.joker_vendor_fee_local + dps.service_fee_local + dps.mov_customer_fee_local AS revenue_local,

      dps.delivery_costs_local,

      (CASE WHEN dps.is_delivery_fee_covered_by_discount = TRUE OR dps.is_delivery_fee_covered_by_voucher = TRUE THEN 0 ELSE dps.delivery_fee_local END) 
      + CASE WHEN dps.entity_id IN ('TB_IQ', 'TB_OM', 'TB_AE', 'TB_KW', 'TB_QA', 'TB_JO', 'TB_BH') THEN tb.commission_amount_lc ELSE dps.commission_local END 
      + dps.joker_vendor_fee_local + dps.service_fee_local + dps.mov_customer_fee_local - dps.delivery_costs_local AS gross_profit_local,

      ROUND(PERCENT_RANK() OVER (ORDER BY travel_time ASC) * 100, 4) tt_percentile,
    FROM `fulfillment-dwh-production.cl.dps_sessions_mapped_to_orders_v2` dps
    INNER JOIN selected_vendors v ON dps.entity_id = v.entity_id AND dps.vendor_id = v.vendor_code
    LEFT JOIN `bta---talabat.data_platform.fct_order` tb ON dps.country_code = LOWER(tb.country_iso) AND dps.platform_order_code = CAST(tb.order_id AS STRING)
    WHERE TRUE 
        -- period
        AND tb.order_date >= {start_date_var}
        AND dps.created_date BETWEEN {start_date_var} AND {end_date_var}
        -- travel time range
        AND dps.travel_time > {min_travel_time_var}
        AND dps.travel_time < {max_travel_time_var}
        -- variants
        AND dps.variant IN UNNEST({included_variants_var})
        -- assignment_type
        AND dps.vendor_price_scheme_type IN UNNEST({included_assignment_types_var})
        AND dps.dps_travel_time_fee_local IN (SELECT DISTINCT fee FROM scheme_config)
)

SELECT 
    created_date,
    platform_order_code,
    vendor_id,
    is_LB,
    travel_time,
    SUM(CASE WHEN travel_time > t.threshold_in_min_and_sec THEN 1 ELSE 0 END) + 1 AS tier,
    dps_travel_time_fee_local,
    delivery_fee_local,
    gfv_local,
    gmv_local,
    commission_local,
    joker_vendor_fee_local,
    mov_customer_fee_local,
    service_fee_local,
    
    revenue_local,
    delivery_costs_local,
    gross_profit_local,
    tt_percentile

FROM order_data ord
CROSS JOIN scheme_config t
GROUP BY 1,2,3,4,5,7,8,9,10,11,12,13,14,15,16,17,18
"""

### Step 2.2: Run the query

In [339]:
client = bigquery.Client(project = 'logistics-data-staging-flat')
bqstorage_client = bigquery_storage.BigQueryReadClient()
df_orders = client.query(sql_query.format(
    entity_id_var = entity_id_var, 
    asa_id_var = asa_id_var, 
    vertical_type_var = vertical_type_var,
    start_date_var = start_date_var,
    end_date_var = end_date_var,
    min_travel_time_var = min_travel_time_var,
    max_travel_time_var = max_travel_time_var,
    included_variants_var = included_variants_var,
    included_assignment_types_var = included_assignment_types_var,
    scheme_id_var = scheme_id_var,
    is_lb_var = is_lb_var
))\
.result()\
.to_dataframe(
    dtypes = {
        'dps_travel_time_fee_local': 'float64', 
        'delivery_fee_local': 'float64', 
        'gfv_local': 'float64', 
        'gmv_local': 'float64', 
        'commission_local': 'float64',
        'joker_vendor_fee_local': 'float64',
        'mov_customer_fee_local': 'float64',
        'service_fee_local': 'float64',
        'revenue_local': 'float64',
        'delivery_costs_local': 'float64',
        'gross_profit_local': 'float64',
        'tt_percentile': 'float64',
    },
    bqstorage_client = bqstorage_client,
    progress_bar_type = 'tqdm_notebook'
)

Downloading:   0%|          | 0/125477 [00:00<?, ?rows/s]

### Step 3.1: Calculating an aggregated data frame by dps_travel_time_fee_local

In [374]:
df_orders_agg = df_orders.groupby(['dps_travel_time_fee_local'])\
    .agg(
        tot_gross_profit_local = pd.NamedAgg(column = 'gross_profit_local', aggfunc = sum),
        tot_df_local = pd.NamedAgg(column = 'delivery_fee_local', aggfunc = sum),
        order_count = pd.NamedAgg(column = 'platform_order_code', aggfunc = len)
    )

df_orders_agg[['avg_gross_profit_local', 'avg_df_local']] = df_orders_agg[['tot_gross_profit_local', 'tot_df_local']].apply(lambda x: round(x / df_orders_agg['order_count'], 4))
df_orders_agg['cum_sum_order_count'] = df_orders_agg['order_count'].cumsum()
df_orders_agg['total_orders'] = df_orders_agg['order_count'].sum()
df_orders_agg['order_share'] = round(df_orders_agg['order_count'] / df_orders_agg['total_orders'], 4)
df_orders_agg['cum_sum_order_share'] = df_orders_agg['order_share'].cumsum()

# Output
current_avg_df = round(sum(df_orders_agg.index.values * df_orders_agg['order_share']), 4)
display(df_orders_agg)
print('The **current** order-weighted TT fee is: {}'.format(current_avg_df))

,tot_gross_profit_local,tot_df_local,order_count,avg_gross_profit_local,avg_df_local,cum_sum_order_count,total_orders,order_share,cum_sum_order_share
dps_travel_time_fee_local,,,,,,,,,
0.65,2518.58630,3797.65,5906,0.4264,0.6430,5906,125477,0.0471,0.0471
0.85,8006.72364,8996.85,10725,0.7465,0.8389,16631,125477,0.0855,0.1326
0.90,7877.20733,7690.70,8744,0.9009,0.8795,25375,125477,0.0697,0.2023
0.95,36552.64834,40086.70,42995,0.8502,0.9324,68370,125477,0.3427,0.5450
1.00,47423.60483,55825.40,57107,0.8304,0.9776,125477,125477,0.4551,1.0001


The **current** order-weighted TT fee is: 0.9467


### Step 3.2: Calculating a new avg DF by reshuffling the order shares

In [382]:
# Define some inputs that will be used from this point onward
# new_order_share = [0.0, 0.0, 0.0, 0.5460, 0.4540] # Remove the first three tiers # INPUT
new_order_share = [0.0, 0.0, 0.0, 0.0, 1.0] # Remove the first three tiers # INPUT
num_variants = 2

df_orders_agg['new_order_share'] = new_order_share
df_orders_agg['cum_sum_new_order_share'] = df_orders_agg['new_order_share'].cumsum()

# Output
new_avg_df = round(sum(df_orders_agg.index.values * df_orders_agg['new_order_share']), 4)
df_pct_uplift = new_avg_df / current_avg_df - 1
display(df_orders_agg)
print('The **new** order-weighted avg DF is: {}. The uplift in avg DF is: {}'\
    .format(
        new_avg_df,        
        str(round(df_pct_uplift * 100, 2)) + '%',        
    )
)

,tot_gross_profit_local,tot_df_local,order_count,avg_gross_profit_local,avg_df_local,cum_sum_order_count,total_orders,order_share,cum_sum_order_share,new_order_share,cum_sum_new_order_share
dps_travel_time_fee_local,,,,,,,,,,,
0.65,2518.58630,3797.65,5906,0.4264,0.6430,5906,125477,0.0471,0.0471,0.0,0.0
0.85,8006.72364,8996.85,10725,0.7465,0.8389,16631,125477,0.0855,0.1326,0.0,0.0
0.90,7877.20733,7690.70,8744,0.9009,0.8795,25375,125477,0.0697,0.2023,0.0,0.0
0.95,36552.64834,40086.70,42995,0.8502,0.9324,68370,125477,0.3427,0.5450,0.0,0.0
1.00,47423.60483,55825.40,57107,0.8304,0.9776,125477,125477,0.4551,1.0001,1.0,1.0


The **new** order-weighted avg DF is: 1.0. The uplift in avg DF is: 5.63%


### Step 4.1: Sample size calculations for per-order KPIs using the Games-Howell test

In [383]:
# Create the data frame that has the means and standard deviation of the per-order KPIs
kpi_list = ['dps_travel_time_fee_local', 'delivery_fee_local', 'gfv_local', 'gmv_local', 'commission_local', 'joker_vendor_fee_local', 'mov_customer_fee_local', 'service_fee_local', 'revenue_local', 'delivery_costs_local', 'gross_profit_local']

def sample_size_in_orders_calc_func(is_lb_bool):
    # Filter for the right orders
    df = df_orders[df_orders['is_LB'].isin(is_lb_bool)]
    
    # Calculate the total days, orders, mean KPIs, and standard deviation of each KPI
    total_days = df['created_date'].nunique()
    total_orders = df['platform_order_code'].nunique()
    df_kpi_means = df[kpi_list].sum() / total_orders
    df_kpi_std_dev = df[kpi_list].std()

    # Create a data frame from the three computations above
    df_kpis = pd.DataFrame(data = {'current_mean': df_kpi_means, 'new_mean': df_kpi_means * (1 + df_pct_uplift), 'std_dev': df_kpi_std_dev})
    df_kpis['effect_size'] = (df_kpis['new_mean'] - df_kpis['current_mean']) / df_kpis['std_dev']
    df_kpis['total_orders'] = total_orders
    df_kpis['is_LB'] = ', '.join(is_lb_bool)
    df_kpis.iloc[:, :-2] = df_kpis.iloc[:, :-2].round(4)

    # Power calculations
    power_analysis = TTestIndPower()

    # Calculate the sample size in **orders** required to reach significance assuming an alpha of 0.05 and a power of 0.8
    sample_size_in_orders = {}
    for i in df_kpis.index.values:
        if pd.isna(df_kpis.loc[i, 'effect_size']):
            sample_size_in_orders[i] = None
        else:
            sample_size = int(round(power_analysis.solve_power(effect_size = df_kpis.loc[i, 'effect_size'], alpha = 0.05, power = 0.8, alternative = 'two-sided'), 0))
            sample_size_in_orders[i] = sample_size

    # Convert the output sample sizes to a series
    sample_size_in_orders = pd.Series(sample_size_in_orders)
    sample_size_in_orders.rename('sample_size_in_orders_per_variant', inplace = True)

    # Merge with df_kpis
    df_kpis = pd.concat([df_kpis, sample_size_in_orders], axis = 1)
    df_kpis['sample_size_in_days_per_variant'] = (df_kpis['sample_size_in_orders_per_variant']) / (df_kpis['total_orders'] / (total_days * num_variants)) # (Total historical orders / num_variants) / Daily avg orders

    # Beautify the output
    df_kpis.dropna(axis = 0, inplace = True) # Remove any records with a 'NA' effect size
    df_kpis['sample_size_in_orders_per_variant'] = df_kpis['sample_size_in_orders_per_variant'].apply('{:,.0f}'.format)
    df_kpis['sample_size_in_days_per_variant'] = df_kpis['sample_size_in_days_per_variant'].apply(ceil).apply('{:,.0f}'.format)
    return df_kpis

In [384]:
# Apply the function
display(sample_size_in_orders_calc_func(['N'])) # 'Y' or 'N' (# INPUT)

,current_mean,new_mean,std_dev,effect_size,total_orders,is_LB,sample_size_in_orders_per_variant,sample_size_in_days_per_variant
dps_travel_time_fee_local,0.9379,0.9907,0.0905,0.5832,64596,N,47,1
delivery_fee_local,0.9198,0.9716,0.1571,0.3297,64596,N,145,1
gfv_local,5.6139,5.9300,4.1968,0.0753,64596,N,"2,769",2
gmv_local,6.3205,6.6764,4.1056,0.0867,64596,N,"2,089",1
commission_local,1.1815,1.2480,0.9422,0.0706,64596,N,"3,150",2
joker_vendor_fee_local,0.1615,0.1706,0.7321,0.0124,64596,N,"102,093",45
revenue_local,2.2562,2.3832,1.2024,0.1056,64596,N,"1,409",1
delivery_costs_local,1.3084,1.3821,0.8099,0.0910,64596,N,"1,897",1
gross_profit_local,0.9406,0.9935,1.4253,0.0372,64596,N,"11,345",5


### Step 4.2: Sample size calculations for per-order and total KPIs using the Wilcoxon-Signed Rank Test

In [385]:
def sample_size_in_days_calc_func(is_lb_bool, totals_or_means):
    # Filter for the right orders
    df = df_orders[df_orders['is_LB'].isin(is_lb_bool)]
    
    # Calculate the total days, as well as the totals per day for each KPI
    total_days = df['created_date'].nunique()
    df_orders_per_day = df.groupby('created_date')['platform_order_code'].nunique()
    df_kpi_per_day = df.groupby('created_date')[kpi_list].sum()
    df_kpi_per_day = pd.concat([df_kpi_per_day, df_orders_per_day], axis = 1)
    df_kpi_per_day.rename(columns={'platform_order_code': 'order_count'}, inplace = True)

    # Calculate the means per day for each KPI
    if totals_or_means == 'totals':
        pass
    else:
        df_kpi_per_day = df_kpi_per_day.iloc[:, :-1].apply(lambda x: x / df_kpi_per_day['order_count'])
    
    # Calculate the standard deviation of df_kpi_per_day
    df_kpi_per_day_agg = pd.DataFrame(
        data = {
            'current_mean_per_day': df_kpi_per_day.mean(), 
            'new_mean_per_day': df_kpi_per_day.mean() * (1 + df_pct_uplift),
            'std_dev_per_day': df_kpi_per_day.std()
        }
    )

    df_kpi_per_day_agg = df_kpi_per_day_agg.assign(
        effect_size = (df_kpi_per_day_agg['new_mean_per_day'] - df_kpi_per_day_agg['current_mean_per_day']) / df_kpi_per_day_agg['std_dev_per_day'],
        total_days = total_days,
        is_LB = ', '.join(is_lb_bool)
    )
    df_kpi_per_day_agg.iloc[:, :-2] = df_kpi_per_day_agg.iloc[:, :-2].round(4)

    # Power calculations
    power_analysis = TTestIndPower()

    sample_size_in_days = {}
    for i in df_kpi_per_day_agg.index.values:
        if pd.isna(df_kpi_per_day_agg.loc[i, 'effect_size']):
            sample_size_in_days[i] = None
        else:
            try:
                sample_size = int(round(power_analysis.solve_power(effect_size = df_kpi_per_day_agg.loc[i, 'effect_size'], alpha = 0.05, power = 0.8, alternative = 'two-sided'), 0))
            except: # The command above will give an error if the standard deviation of the metric is too small. Use a try-except construct to mitigate that
                sample_size = 7 # Minimum experiment duration of 1 week if the standard deviation of the metric is too small, generating an equally small sample size in days    
            sample_size_in_days[i] = sample_size

    # Convert the output sample sizes to a series
    sample_size_in_days = pd.Series(sample_size_in_days)
    sample_size_in_days.rename('sample_size_in_days_per_variant', inplace = True)

    # Merge with df_kpis
    df_kpi_per_day_agg = pd.concat([df_kpi_per_day_agg, sample_size_in_days], axis = 1)

    # Beautify the output
    df_kpi_per_day_agg.dropna(axis = 0, inplace = True) # Remove any records with a 'NA' effect size
    df_kpi_per_day_agg['sample_size_in_days_per_variant'] = df_kpi_per_day_agg['sample_size_in_days_per_variant'].apply(ceil).apply('{:,.0f}'.format)

    return df_kpi_per_day, df_kpi_per_day_agg

In [388]:
# Apply the function
df_kpi_per_day, df_kpi_per_day_agg = sample_size_in_days_calc_func(['Y'], 'totals') # 'Y' or 'N'; 'totals' or 'means' (# INPUT)

# Output
display(df_kpi_per_day)
display(df_kpi_per_day_agg)

,dps_travel_time_fee_local,delivery_fee_local,gfv_local,gmv_local,commission_local,joker_vendor_fee_local,mov_customer_fee_local,service_fee_local,revenue_local,delivery_costs_local,gross_profit_local,order_count
created_date,,,,,,,,,,,,
2022-09-04,7309.25,7161.50,39080.39,45099.45,8266.27384,830.39,0.00,0.0,16212.41384,10867.674,5296.09064,7721
2022-09-05,8034.00,7884.40,42632.50,49260.45,9042.02506,850.28,0.00,0.0,17708.20506,11941.790,5704.93056,8520
2022-09-06,8266.85,8104.85,44460.03,51366.24,9470.00033,872.64,0.00,0.0,18386.84033,12221.127,6113.74633,8735
2022-09-07,8487.30,8325.30,45402.88,52408.77,9686.50182,911.39,0.00,0.0,18856.64182,11958.404,6841.58032,8957
2022-09-08,8732.65,8573.40,50380.26,57529.05,10640.79849,998.05,0.00,0.0,20144.64849,11925.871,8175.01949,9225
2022-09-09,9588.15,9382.15,56453.64,64341.00,12028.18825,1143.49,0.00,0.0,22485.77825,11848.325,10584.90525,10097
2022-09-10,8536.25,8375.95,45977.88,53074.36,9800.23071,962.96,0.00,0.0,19076.84071,11827.590,7198.02871,8999
2022-09-11,8141.40,7996.50,42540.18,49406.92,9045.22476,801.55,0.00,0.0,17789.67476,11755.858,5986.47076,8597
2022-09-12,8189.95,8025.40,42903.53,49732.42,9164.28490,875.10,0.00,0.0,18011.43490,11353.341,6610.86790,8660


,current_mean_per_day,new_mean_per_day,std_dev_per_day,effect_size,total_days,is_LB,sample_size_in_days_per_variant
dps_travel_time_fee_local,8484.0714,8961.7317,548.1759,0.8714,14,"Y, N",22
delivery_fee_local,8314.0929,8782.1832,531.3359,0.8810,14,"Y, N",21
gfv_local,46528.2864,49147.8678,4823.8786,0.5430,14,"Y, N",54
gmv_local,53575.1707,56591.4975,5231.6650,0.5766,14,"Y, N",48
commission_local,9898.3015,10455.5841,1015.8263,0.5486,14,"Y, N",53
joker_vendor_fee_local,941.0950,994.0794,122.5818,0.4322,14,"Y, N",85
mov_customer_fee_local,1.9107,2.0183,3.8141,0.0282,14,"Y, N","19,741"
revenue_local,19099.4000,20174.7122,1645.0772,0.6537,14,"Y, N",38
delivery_costs_local,11732.8696,12393.4400,370.2765,1.7840,14,"Y, N",6
gross_profit_local,7312.7693,7724.4843,1533.9794,0.2684,14,"Y, N",219
